In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor, plot_tree
from sklearn.tree import export_graphviz
import graphviz

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn import preprocessing

import gc
import xgboost as xgb
RANDOM_STATE=42

pd.options.display.max_columns = None


In [ ]:
# reduces the memory of a dataframe
def reduce_mem_usage(df, verbose = True):
    
    '''
    Reduces the space that a DataFrame occupies in memory.

    This function iterates over all columns in a df and downcasts them to lower type to save memory.
    '''
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'\
              .format(end_mem, 100 * (start_mem - end_mem) / start_mem))

# Unimos Transacciones y clientes

In [2]:
trans=pd.read_pickle('./3T/itau_fe.pkl',compression='zip')

In [3]:
consumidores=pd.read_pickle('./3T/Consumidores_v2.pkl', compression='zip')

In [5]:
consumidores.head()

,Edad,Sexo,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,id,EdadNumber,RentaNum,Segmento_consumidorNum,Meses_antiguedadNum,Estado_civil_A,Estado_civil_B,Estado_civil_C,Estado_civil_D,Estado_civil_E,Principalidad_A,Principalidad_B,Principalidad_C,Principalidad_D,Principalidad_E,Principalidad_F,Principalidad_G,Principalidad_XXX
0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,1,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0
1,Mayor a 70,1,R9,0,B,Mayor a 10 años,37.0,5.0,D,E,P164,2,80,9,1,10,0,0,0,1,0,0,0,0,0,1,0,0,0
2,Mayor a 70,1,R9,1,B,Mayor a 10 años,19.0,4.0,B,D,P114,3,80,9,1,10,0,1,0,0,0,0,0,0,1,0,0,0,0
3,Mayor a 70,1,R10,0,B,Mayor a 10 años,37.0,5.0,B,C,P164,4,80,10,1,10,0,1,0,0,0,0,0,1,0,0,0,0,0
4,Mayor a 70,1,R4,0,A,Mayor a 10 años,91.0,13.0,D,B,P85,5,80,4,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0


In [4]:
consumidores.isnull().sum()

Edad                       0
Sexo                       0
Renta                      0
Recibe_sueldo_en_cuenta    0
Segmento_consumidor        0
Meses_antiguedad           0
Comuna                     0
Ciudad                     0
Estado_civil               0
Principalidad              0
Profesion                  0
id                         0
EdadNumber                 0
RentaNum                   0
Segmento_consumidorNum     0
Meses_antiguedadNum        0
Estado_civil_A             0
Estado_civil_B             0
Estado_civil_C             0
Estado_civil_D             0
Estado_civil_E             0
Principalidad_A            0
Principalidad_B            0
Principalidad_C            0
Principalidad_D            0
Principalidad_E            0
Principalidad_F            0
Principalidad_G            0
Principalidad_XXX          0
dtype: int64

In [5]:
trans.isnull().sum().sum()

0

## Hacemos el Merge trans con cliente

In [6]:
len(trans),len(consumidores)

(8908368, 79539)

In [7]:
df = pd.merge(trans,consumidores , on = ['id'], how = 'left')

In [12]:
df

,id,Periodo,Producto-Tipo,Count,MontoSignoSum,MontoSignoMean,MontoSignoMax,MontoSignoMin,NumProductos,NumProductosGeneral,Count-1,Count-2,Count-3,Count-4,Count-5,MontoSignoSum-1,MontoSignoSum-2,MontoSignoSum-3,MontoSignoSum-4,MontoSignoSum-5,MontoSignoMean-1,MontoSignoMean-2,MontoSignoMean-3,MontoSignoMean-4,MontoSignoMean-5,MontoSignoMax-1,MontoSignoMax-2,MontoSignoMax-3,MontoSignoMax-4,MontoSignoMax-5,MontoSignoMin-1,MontoSignoMin-2,MontoSignoMin-3,MontoSignoMin-4,MontoSignoMin-5,NumProductos-1,NumProductos-2,NumProductos-3,NumProductos-4,NumProductos-5,NumProductosGeneral-1,NumProductosGeneral-2,NumProductosGeneral-3,NumProductosGeneral-4,NumProductosGeneral-5,year,month,Producto-Tipo_A-A,Producto-Tipo_A-G,Producto-Tipo_A-K,Producto-Tipo_A-L,Producto-Tipo_B-B,Producto-Tipo_C-C,Producto-Tipo_C-D,Producto-Tipo_D-E,Producto-Tipo_D-F,Producto-Tipo_E-E,Producto-Tipo_E-F,Producto-Tipo_F-D,Producto-Tipo_F-H,Producto-Tipo_F-I,Producto-Tipo_F-J,Producto-Tipo_G-K,Edad,Sexo,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,EdadNumber,RentaNum,Segmento_consumidorNum,Meses_antiguedadNum,Estado_civil_A,Estado_civil_B,Estado_civil_C,Estado_civil_D,Estado_civil_E,Principalidad_A,Principalidad_B,Principalidad_C,Principalidad_D,Principalidad_E,Principalidad_F,Principalidad_G,Principalidad_XXX
0,1,2019-04-01,A-G,4.0,1015275.000,253818.75000,281602.000000,2.295660e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0
1,1,2019-04-01,A-K,76.0,-9978991.000,-131302.50000,-121428.570312,-1.733537e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0
2,1,2019-04-01,F-D,3.0,1417142.875,472380.96875,577142.875000,2.914286e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0
3,1,2019-04-01,F-H,3.0,-1417142.875,-472380.96875,-291428.562500,-5.771429e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0
4,1,2019-04-01,F-J,3.0,1417142.875,472380.96875,577142.875000,2.914286e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0

In [8]:
df.isnull().sum().sum()

0

In [9]:
# Ponemos todo a uno
df.loc[ df['Count']>0 ,'Count']=1

## Vemos las variables Profesion Ciudad y Comuna

In [8]:
df[['id','Periodo','Producto-Tipo','Count','Profesion','Ciudad','Comuna']]

,id,Periodo,Producto-Tipo,Count,Profesion,Ciudad,Comuna
0,1,2019-04-01,A-G,1.0,P164,13.0,331.0
1,1,2019-04-01,A-K,1.0,P164,13.0,331.0
2,1,2019-04-01,F-D,1.0,P164,13.0,331.0
3,1,2019-04-01,F-H,1.0,P164,13.0,331.0
4,1,2019-04-01,F-J,1.0,P164,13.0,331.0
...,...,...,...,...,...,...,...
8908363,79539,2020-10-01,C-C,1.0,PXXX,5.0,34.0
8908364,79539,2020-10-01,E-F,1.0,PXXX,5.0,34.0
8908365,79539,2020-10-01,B-B,0.0,PXXX,5.0,34.0
8908366,79539,2020-10-01,A-A,0.0,PXXX,5.0,34.0


In [30]:
# Profesion 
# Hacemos un frecuency encoder y un mean encoder

In [12]:
df['Profesion'].value_counts(dropna=False)

P164    1111376
P227     797440
P84      629328
P85      603456
P149     520240
         ...   
P202        112
P200        112
P132        112
P128        112
P119        112
Name: Profesion, Length: 211, dtype: int64

In [9]:
# MeanEncoder
variable='Profesion'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo'])['Count'].mean()
stage=stage.reset_index(drop=False, name=variable+'_Meanratio')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [31]:
df['Profesion_Meanratio'].isnull().sum()

0

In [14]:
# MeanEncoder
#variable='Profesion'
#ratio_encoder=df[ ((df['Producto-Tipo']=='A-A') |
#                  (df['Producto-Tipo']=='B-B') |
#                  (df['Producto-Tipo']=='C-D') |
#                  (df['Producto-Tipo']=='D-E') |
#                  (df['Producto-Tipo']=='E-E')) & 
#                  (df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable])['Count'].mean()
#df.loc[:,variable+'_Meanratio']=df[variable].map(ratio_encoder)

In [15]:
df[['id','Periodo','Producto-Tipo','Count','Profesion','Profesion_Meanratio']]

,id,Periodo,Producto-Tipo,Count,Profesion,Profesion_Meanratio
0,1,2019-04-01,A-G,1.0,P164,0.121216
1,1,2019-04-01,A-K,1.0,P164,0.121216
2,1,2019-04-01,F-D,1.0,P164,0.121216
3,1,2019-04-01,F-H,1.0,P164,0.121216
4,1,2019-04-01,F-J,1.0,P164,0.121216
...,...,...,...,...,...,...
8908363,79539,2020-10-01,C-C,1.0,PXXX,0.148926
8908364,79539,2020-10-01,E-F,1.0,PXXX,0.148926
8908365,79539,2020-10-01,B-B,0.0,PXXX,0.148926
8908366,79539,2020-10-01,A-A,0.0,PXXX,0.148926


In [41]:
df['Profesion_Meanratio'].value_counts()

0.103882    1111376
0.098022     797440
0.136963     629328
0.133789     603456
0.176147     520240
             ...   
0.295166        336
0.314209        112
0.171387        112
0.199951        112
0.371338        112
Name: Profesion_Meanratio, Length: 171, dtype: int64

In [10]:
# frecuency encoder
variable='Profesion'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo']).size()/len(df)
stage=stage.reset_index(drop=False, name=variable+'_frec')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [16]:
# frecuency encoder
#variable='Profesion'
#fe=df[ ((df['Producto-Tipo']=='A-A') |
#                  (df['Producto-Tipo']=='B-B') |
#                  (df['Producto-Tipo']=='C-D') |
#                  (df['Producto-Tipo']=='D-E') |
#                  (df['Producto-Tipo']=='E-E')) & 
#                  (df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable]).size()/len(df)
#df.loc[:,variable+'_frec']=df[variable].map(fe)


In [33]:
df[variable+'_frec'].isnull().sum()

0

In [11]:
df[['id','Periodo','Producto-Tipo','Count','Profesion','Profesion_Meanratio','Profesion_frec']]

,id,Periodo,Producto-Tipo,Count,Profesion,Profesion_Meanratio,Profesion_frec
0,1,2019-04-01,A-G,1.0,P164,0.879395,0.006683
1,1,2019-04-01,A-K,1.0,P164,0.827637,0.006683
2,1,2019-04-01,F-D,1.0,P164,0.350342,0.006683
3,1,2019-04-01,F-H,1.0,P164,0.679688,0.006683
4,1,2019-04-01,F-J,1.0,P164,0.899902,0.006683
...,...,...,...,...,...,...,...
8908363,79539,2020-10-01,C-C,1.0,PXXX,0.239502,0.000284
8908364,79539,2020-10-01,E-F,1.0,PXXX,0.211426,0.000284
8908365,79539,2020-10-01,B-B,0.0,PXXX,0.104919,0.000284
8908366,79539,2020-10-01,A-A,0.0,PXXX,0.007919,0.000284


In [12]:
df[['id','Periodo','Producto-Tipo','Count','Profesion','Profesion_Meanratio','Profesion_frec']].isnull().sum()

id                     0
Periodo                0
Producto-Tipo          0
Count                  0
Profesion              0
Profesion_Meanratio    0
Profesion_frec         0
dtype: int64

In [ ]:
# Ciudad
# Hacemos un frecuency encoder y un mean encoder

In [19]:
df['Ciudad'].value_counts(dropna=False)

 13.0     5241824
 8.0      1070720
 5.0       694400
 2.0       266896
 7.0       264768
 10.0      246848
 4.0       211344
 6.0       207200
 9.0       194208
 1.0       178528
 3.0       112784
 12.0       98784
 14.0       80528
 15.0       34608
 11.0        3360
 16.0        1120
-999.0        448
Name: Ciudad, dtype: int64

In [20]:
# MeanEncoder
variable='Ciudad'
ratio_encoder=df[ ((df['Producto-Tipo']=='A-A') |
                  (df['Producto-Tipo']=='B-B') |
                  (df['Producto-Tipo']=='C-D') |
                  (df['Producto-Tipo']=='D-E') |
                  (df['Producto-Tipo']=='E-E')) & 
                  (df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable])['Count'].mean()
df.loc[:,variable+'_Meanratio']=df[variable].map(ratio_encoder)

In [21]:
# frecuency encoder
variable='Ciudad'
fe=df[ ((df['Producto-Tipo']=='A-A') |
                  (df['Producto-Tipo']=='B-B') |
                  (df['Producto-Tipo']=='C-D') |
                  (df['Producto-Tipo']=='D-E') |
                  (df['Producto-Tipo']=='E-E')) & 
                  (df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable]).size()/len(df)
df.loc[:,variable+'_frec']=df[variable].map(fe)


In [11]:
# MeanEncoder
variable='Ciudad'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo'])['Count'].mean()
stage=stage.reset_index(drop=False, name=variable+'_Meanratio')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [12]:
# frecuency encoder
variable='Ciudad'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo']).size()/len(df)
stage=stage.reset_index(drop=False, name=variable+'_frec')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [15]:
df[['id','Periodo','Producto-Tipo','Count','Ciudad','Ciudad_Meanratio','Ciudad_frec']]

,id,Periodo,Producto-Tipo,Count,Ciudad,Ciudad_Meanratio,Ciudad_frec
0,1,2019-04-01,A-G,1.0,13.0,0.882324,0.031522
1,1,2019-04-01,A-K,1.0,13.0,0.842773,0.031522
2,1,2019-04-01,F-D,1.0,13.0,0.328857,0.031522
3,1,2019-04-01,F-H,1.0,13.0,0.708496,0.031522
4,1,2019-04-01,F-J,1.0,13.0,0.909668,0.031522
...,...,...,...,...,...,...,...
8908363,79539,2020-10-01,C-C,1.0,5.0,0.177734,0.004176
8908364,79539,2020-10-01,E-F,1.0,5.0,0.160767,0.004176
8908365,79539,2020-10-01,B-B,0.0,5.0,0.112610,0.004176
8908366,79539,2020-10-01,A-A,0.0,5.0,0.029221,0.004176


In [16]:
df[['id','Periodo','Producto-Tipo','Count','Ciudad','Ciudad_Meanratio','Ciudad_frec']].isnull().sum()

id                  0
Periodo             0
Producto-Tipo       0
Count               0
Ciudad              0
Ciudad_Meanratio    0
Ciudad_frec         0
dtype: int64

In [52]:
# Comuna
# Hacemos un frecuency encoder y un mean encoder

In [24]:
df['Comuna'].value_counts(dropna=False)

71.0      718928
70.0      660016
91.0      495376
72.0      390208
188.0     370832
           ...  
177.0        112
226.0        112
223.0        112
187.0        112
1920.0       112
Name: Comuna, Length: 323, dtype: int64

In [25]:
# MeanEncoder
variable='Comuna'
ratio_encoder=df[ ((df['Producto-Tipo']=='A-A') |
                  (df['Producto-Tipo']=='B-B') |
                  (df['Producto-Tipo']=='C-D') |
                  (df['Producto-Tipo']=='D-E') |
                  (df['Producto-Tipo']=='E-E')) & 
                  (df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable])['Count'].mean()
df.loc[:,variable+'_Meanratio']=df[variable].map(ratio_encoder)

In [26]:
# frecuency encoder
variable='Comuna'
fe=df[ ((df['Producto-Tipo']=='A-A') |
                  (df['Producto-Tipo']=='B-B') |
                  (df['Producto-Tipo']=='C-D') |
                  (df['Producto-Tipo']=='D-E') |
                  (df['Producto-Tipo']=='E-E')) & 
                  (df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable]).size()/len(df)
df.loc[:,variable+'_frec']=df[variable].map(fe)


In [13]:
# MeanEncoder
variable='Comuna'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo'])['Count'].mean()
stage=stage.reset_index(drop=False, name=variable+'_Meanratio')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [14]:
# frecuency encoder
variable='Comuna'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo']).size()/len(df)
stage=stage.reset_index(drop=False, name=variable+'_frec')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [19]:
df[['id','Periodo','Producto-Tipo','Count','Comuna','Comuna_Meanratio','Comuna_frec']]

,id,Periodo,Producto-Tipo,Count,Comuna,Comuna_Meanratio,Comuna_frec
0,1,2019-04-01,A-G,1.0,331.0,0.942871,0.001004
1,1,2019-04-01,A-K,1.0,331.0,0.916992,0.001004
2,1,2019-04-01,F-D,1.0,331.0,0.450195,0.001004
3,1,2019-04-01,F-H,1.0,331.0,0.708008,0.001004
4,1,2019-04-01,F-J,1.0,331.0,0.940918,0.001004
...,...,...,...,...,...,...,...
8908363,79539,2020-10-01,C-C,1.0,34.0,0.184692,0.000846
8908364,79539,2020-10-01,E-F,1.0,34.0,0.160400,0.000846
8908365,79539,2020-10-01,B-B,0.0,34.0,0.101807,0.000846
8908366,79539,2020-10-01,A-A,0.0,34.0,0.025208,0.000846


In [15]:
df[['id','Periodo','Producto-Tipo','Count','Comuna','Comuna_Meanratio','Comuna_frec']].isnull().sum()

id                  0
Periodo             0
Producto-Tipo       0
Count               0
Comuna              0
Comuna_Meanratio    0
Comuna_frec         0
dtype: int64

In [16]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8908368 entries, 0 to 8908367
Data columns (total 80 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   id                       int32         
 1   Periodo                  datetime64[ns]
 2   Producto-Tipo            object        
 3   Count                    float16       
 4   MontoSignoSum            float32       
 5   MontoSignoMean           float32       
 6   MontoSignoMax            float32       
 7   MontoSignoMin            float32       
 8   Count-1                  float16       
 9   Count-2                  float16       
 10  Count-3                  float16       
 11  Count-4                  float16       
 12  MontoSignoSum-1          float32       
 13  MontoSignoSum-2          float32       
 14  MontoSignoSum-3          float32       
 15  MontoSignoSum-4          float32       
 16  MontoSignoMean-1         float32       
 17  MontoSignoMean-2         fl

In [17]:
# Para estado civil
df['Estado_civil'].value_counts()

A    5131504
B    2334080
D     946176
E     417648
C      78960
Name: Estado_civil, dtype: int64

In [18]:
# MeanEncoder
variable='Estado_civil'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo'])['Count'].mean()
stage=stage.reset_index(drop=False, name=variable+'_Meanratio')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [19]:
# frecuency encoder
variable='Estado_civil'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo']).size()/len(df)
stage=stage.reset_index(drop=False, name=variable+'_frec')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [24]:
df[['id','Periodo','Producto-Tipo','Count','Estado_civil','Estado_civil_Meanratio','Estado_civil_frec']].isnull().sum()

id                        0
Periodo                   0
Producto-Tipo             0
Count                     0
Estado_civil              0
Estado_civil_Meanratio    0
Estado_civil_frec         0
dtype: int64

In [20]:
# Para Principalidad
df['Principalidad'].value_counts()

B      2017904
A      1734544
XXX    1362032
F      1098832
D      1050336
E       599760
G       561232
C       483728
Name: Principalidad, dtype: int64

In [21]:
# MeanEncoder
variable='Principalidad'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo'])['Count'].mean()
stage=stage.reset_index(drop=False, name=variable+'_Meanratio')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [22]:
# frecuency encoder
variable='Principalidad'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo']).size()/len(df)
stage=stage.reset_index(drop=False, name=variable+'_frec')
df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')

In [23]:
df[['id','Periodo','Producto-Tipo','Count','Principalidad','Principalidad_Meanratio','Principalidad_frec']].isnull().sum()

id                         0
Periodo                    0
Producto-Tipo              0
Count                      0
Principalidad              0
Principalidad_Meanratio    0
Principalidad_frec         0
dtype: int64

# Generamos los encoders

In [10]:
for x in ['Sexo','Recibe_sueldo_en_cuenta','Comuna','Ciudad','Estado_civil','Principalidad','Profesion','EdadNumber',                 
            'RentaNum','Segmento_consumidorNum','Meses_antiguedadNum']:
    print('Vamos a generar para:'+x)
    # MeanEncoder
    print('MeanEncoder')
    variable=x
    stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo'])['Count'].mean()
    stage=stage.reset_index(drop=False, name=variable+'_Meanratio')
    df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')
    # frecuency encoder
    print('frecuency encoder')
    stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby([variable,'Producto-Tipo']).size()/len(df)
    stage=stage.reset_index(drop=False, name=variable+'_frec')
    df = pd.merge(df,stage , on = [variable,'Producto-Tipo'], how = 'left')


Vamos a generar para:Sexo
MeanEncoder
frecuency encoder
Vamos a generar para:Recibe_sueldo_en_cuenta
MeanEncoder
frecuency encoder
Vamos a generar para:Comuna
MeanEncoder
frecuency encoder
Vamos a generar para:Ciudad
MeanEncoder
frecuency encoder
Vamos a generar para:Estado_civil
MeanEncoder
frecuency encoder
Vamos a generar para:Principalidad
MeanEncoder
frecuency encoder
Vamos a generar para:Profesion
MeanEncoder
frecuency encoder
Vamos a generar para:EdadNumber
MeanEncoder
frecuency encoder
Vamos a generar para:RentaNum
MeanEncoder
frecuency encoder
Vamos a generar para:Segmento_consumidorNum
MeanEncoder
frecuency encoder
Vamos a generar para:Meses_antiguedadNum
MeanEncoder
frecuency encoder


In [16]:
df[['id','Periodo','Producto-Tipo','Count','Principalidad','Principalidad_Meanratio','Principalidad_frec']]

,id,Periodo,Producto-Tipo,Count,Principalidad,Principalidad_Meanratio,Principalidad_frec
0,1,2019-04-01,A-G,1.0,B,0.860352,0.012135
1,1,2019-04-01,A-K,1.0,B,0.813965,0.012135
2,1,2019-04-01,F-D,1.0,B,0.274902,0.012135
3,1,2019-04-01,F-H,1.0,B,0.608398,0.012135
4,1,2019-04-01,F-J,1.0,B,0.872559,0.012135
...,...,...,...,...,...,...,...
8908363,79539,2020-10-01,C-C,1.0,XXX,0.512695,0.008191
8908364,79539,2020-10-01,E-F,1.0,XXX,0.158936,0.008191
8908365,79539,2020-10-01,B-B,0.0,XXX,0.151855,0.008191
8908366,79539,2020-10-01,A-A,0.0,XXX,0.034637,0.008191


In [17]:
#df[['id','Periodo','Producto-Tipo','Count','id','id_Meanratio','id_frec']]

# Cargamos campañas

In [11]:
camps=pd.read_pickle('./3T/itau_campana_fe.pkl',compression='zip')

In [12]:
df = pd.merge(df,camps , on = ['id','Periodo','Producto-Tipo'], how = 'left')

# Cargamos Notificaciones

In [13]:
comu=pd.read_pickle('./3T/itau_comunicaciones_fe.pkl',compression='zip')

In [14]:
df = pd.merge(df,comu , on = ['id','Periodo','Producto-Tipo'], how = 'left')

In [15]:
df

,id,Periodo,Producto-Tipo,Count,MontoSignoSum,MontoSignoMean,MontoSignoMax,MontoSignoMin,NumProductos,NumProductosGeneral,Count-1,Count-2,Count-3,Count-4,Count-5,MontoSignoSum-1,MontoSignoSum-2,MontoSignoSum-3,MontoSignoSum-4,MontoSignoSum-5,MontoSignoMean-1,MontoSignoMean-2,MontoSignoMean-3,MontoSignoMean-4,MontoSignoMean-5,MontoSignoMax-1,MontoSignoMax-2,MontoSignoMax-3,MontoSignoMax-4,MontoSignoMax-5,MontoSignoMin-1,MontoSignoMin-2,MontoSignoMin-3,MontoSignoMin-4,MontoSignoMin-5,NumProductos-1,NumProductos-2,NumProductos-3,NumProductos-4,NumProductos-5,NumProductosGeneral-1,NumProductosGeneral-2,NumProductosGeneral-3,NumProductosGeneral-4,NumProductosGeneral-5,year,month,Producto-Tipo_A-A,Producto-Tipo_A-G,Producto-Tipo_A-K,Producto-Tipo_A-L,Producto-Tipo_B-B,Producto-Tipo_C-C,Producto-Tipo_C-D,Producto-Tipo_D-E,Producto-Tipo_D-F,Producto-Tipo_E-E,Producto-Tipo_E-F,Producto-Tipo_F-D,Producto-Tipo_F-H,Producto-Tipo_F-I,Producto-Tipo_F-J,Producto-Tipo_G-K,Edad,Sexo,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,EdadNumber,RentaNum,Segmento_consumidorNum,Meses_antiguedadNum,Estado_civil_A,Estado_civil_B,Estado_civil_C,Estado_civil_D,Estado_civil_E,Principalidad_A,Principalidad_B,Principalidad_C,Principalidad_D,Principalidad_E,Principalidad_F,Principalidad_G,Principalidad_XXX,Sexo_Meanratio,Sexo_frec,Recibe_sueldo_en_cuenta_Meanratio,Recibe_sueldo_en_cuenta_frec,Comuna_Meanratio,Comuna_frec,Ciudad_Meanratio,Ciudad_frec,Estado_civil_Meanratio,Estado_civil_frec,Principalidad_Meanratio,Principalidad_frec,Profesion_Meanratio,Profesion_frec,EdadNumber_Meanratio,EdadNumber_frec,RentaNum_Meanratio,RentaNum_frec,Segmento_consumidorNum_Meanratio,Segmento_consumidorNum_frec,Meses_antiguedadNum_Meanratio,Meses_antiguedadNum_frec,Count-campana,Canal_B,Canal_C,Duracion_campana,Resultado,Count-campana-1,Count-campana-2,Count-campana-3,Canal_B-1,Canal_B-2,Canal_B-3,Canal_C-1,Canal_C-2,Canal_C-3,Duracion_campana-1,Duracion_campana-2,Duracion_campana-3,Resultado-1,Resultado-2,Resultado-3,Count-comu,Lectura,LecturaMean,LecturaMax,LecturaMin,Count-comu-1,Count-comu-2,Count-comu-3,Count-comu-4,Count-comu-5,Lectura-1,Lectura-2,Lectura-3,Lectura-4,Lectura-5,LecturaMean-1,LecturaMean-2,LecturaMean-3,LecturaMean-4,LecturaMean-5,LecturaMax-1,LecturaMax-2,LecturaMax-3,LecturaMax-4,LecturaMax-5,LecturaMin-1,LecturaMin-2,LecturaMin-3,LecturaMin-4,LecturaMin-5
0,1,2019-04-01,A-G,1.0,1015275.000,253818.75000,281602.000000,2.295660e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0,0.883789,0.031964,0.866699,0.02967,0.942871,0.001004,0.882324,0.031522,0.884766,0.00569,0.860352,0.012135,0.879395,0.006683,0.924805,0.001966,0.911133,0.006025,0.855957,0.039681,0.934082,0.001204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2019-04-01,A-K,1.0,-9978991.000,-131302.50000,-121428.570312,-1.733537e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0,0.836914,0.031964,0.818359,0.02967,0.916992,0.001004,0.842773,0.031522,0.816406,0.00569,0.813965,0

In [16]:
df.isnull().sum()

id                     0
Periodo                0
Producto-Tipo          0
Count                  0
MontoSignoSum          0
                  ...   
LecturaMin-1     6189043
LecturaMin-2     6189043
LecturaMin-3     6189043
LecturaMin-4     6189043
LecturaMin-5     6189043
Length: 163, dtype: int64

In [17]:
df.fillna(-999, inplace=True)

In [18]:
df.isnull().sum()

id               0
Periodo          0
Producto-Tipo    0
Count            0
MontoSignoSum    0
                ..
LecturaMin-1     0
LecturaMin-2     0
LecturaMin-3     0
LecturaMin-4     0
LecturaMin-5     0
Length: 163, dtype: int64

In [19]:
reduce_mem_usage(df)

NameError: name 'reduce_mem_usage' is not defined

In [26]:
#df.to_pickle('./3T/df_trans_campanas_comu_3M_V5.pkl',compression='zip')
#df.to_pickle('./3T/df_trans_campanas_comu_3M_V6.pkl',compression='zip')
df.to_pickle('./3T/df_trans_campanas_comu_3M_V7.pkl',compression='zip')


In [25]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8908368 entries, 0 to 8908367
Data columns (total 163 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   id                                 int32         
 1   Periodo                            datetime64[ns]
 2   Producto-Tipo                      object        
 3   Count                              float16       
 4   MontoSignoSum                      float32       
 5   MontoSignoMean                     float32       
 6   MontoSignoMax                      float32       
 7   MontoSignoMin                      float32       
 8   NumProductos                       float16       
 9   NumProductosGeneral                float16       
 10  Count-1                            float16       
 11  Count-2                            float16       
 12  Count-3                            float16       
 13  Count-4                            float16       
 14  C

In [74]:
df=pd.read_pickle('./3T/df_trans_campanas_comu_3M_V6.pkl',compression='zip')

In [5]:
variable='Producto-Tipo'
stage=df[(df['Periodo']!=pd.to_datetime('202010',format='%Y%m'))].groupby(['Producto-Tipo'])['Count'].mean()
stage=stage.reset_index(drop=False, name=variable+'_Meanratio')
df = pd.merge(df,stage , on = ['Producto-Tipo'], how = 'left')

In [6]:
df


,id,Periodo,Producto-Tipo,Count,MontoSignoSum,MontoSignoMean,MontoSignoMax,MontoSignoMin,NumProductos,NumProductosGeneral,Count-1,Count-2,Count-3,Count-4,Count-5,MontoSignoSum-1,MontoSignoSum-2,MontoSignoSum-3,MontoSignoSum-4,MontoSignoSum-5,MontoSignoMean-1,MontoSignoMean-2,MontoSignoMean-3,MontoSignoMean-4,MontoSignoMean-5,MontoSignoMax-1,MontoSignoMax-2,MontoSignoMax-3,MontoSignoMax-4,MontoSignoMax-5,MontoSignoMin-1,MontoSignoMin-2,MontoSignoMin-3,MontoSignoMin-4,MontoSignoMin-5,NumProductos-1,NumProductos-2,NumProductos-3,NumProductos-4,NumProductos-5,NumProductosGeneral-1,NumProductosGeneral-2,NumProductosGeneral-3,NumProductosGeneral-4,NumProductosGeneral-5,year,month,Producto-Tipo_A-A,Producto-Tipo_A-G,Producto-Tipo_A-K,Producto-Tipo_A-L,Producto-Tipo_B-B,Producto-Tipo_C-C,Producto-Tipo_C-D,Producto-Tipo_D-E,Producto-Tipo_D-F,Producto-Tipo_E-E,Producto-Tipo_E-F,Producto-Tipo_F-D,Producto-Tipo_F-H,Producto-Tipo_F-I,Producto-Tipo_F-J,Producto-Tipo_G-K,Edad,Sexo,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,EdadNumber,RentaNum,Segmento_consumidorNum,Meses_antiguedadNum,Estado_civil_A,Estado_civil_B,Estado_civil_C,Estado_civil_D,Estado_civil_E,Principalidad_A,Principalidad_B,Principalidad_C,Principalidad_D,Principalidad_E,Principalidad_F,Principalidad_G,Principalidad_XXX,Sexo_Meanratio,Sexo_frec,Recibe_sueldo_en_cuenta_Meanratio,Recibe_sueldo_en_cuenta_frec,Comuna_Meanratio,Comuna_frec,Ciudad_Meanratio,Ciudad_frec,Estado_civil_Meanratio,Estado_civil_frec,Principalidad_Meanratio,Principalidad_frec,Profesion_Meanratio,Profesion_frec,EdadNumber_Meanratio,EdadNumber_frec,RentaNum_Meanratio,RentaNum_frec,Segmento_consumidorNum_Meanratio,Segmento_consumidorNum_frec,Meses_antiguedadNum_Meanratio,Meses_antiguedadNum_frec,Count-campana,Canal_B,Canal_C,Duracion_campana,Resultado,Count-campana-1,Count-campana-2,Count-campana-3,Canal_B-1,Canal_B-2,Canal_B-3,Canal_C-1,Canal_C-2,Canal_C-3,Duracion_campana-1,Duracion_campana-2,Duracion_campana-3,Resultado-1,Resultado-2,Resultado-3,Count-comu,Lectura,Count-comu-1,Count-comu-2,Count-comu-3,Count-comu-4,Count-comu-5,Lectura-1,Lectura-2,Lectura-3,Lectura-4,Lectura-5,Producto-Tipo_Meanratio
0,1,2019-04-01,A-G,1.0,1015275.000,253818.75000,281602.000000,2.295660e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0,0.883789,0.031952,0.866699,0.029663,0.942871,0.001004,0.882324,0.031525,0.884766,0.005692,0.860352,0.012138,0.879395,0.006683,0.924805,0.001966,0.911133,0.006027,0.855957,0.039673,0.934082,0.001204,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.876953
1,1,2019-04-01,A-K,1.0,-9978991.000,-131302.50000,-121428.570312,-1.733537e+05,0.0,6.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0000,-999.0000,-999.000,-999.00,-999.0,-999.00000,-999.00000,-999.000000,-999.000,-999.0000,-999.00000,-999.000000,-999.000000,-999.0000,-999.0000,-999.000000,-999.000000,-9.990000e+02,-999.0000,-999.0000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2019,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Mayor a 70,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,80,1,0,10,0,0,0,1,0,0,1,0,0,0,0,0,0,0.836914,0.031952,0.818359,0.029663,0.916992,0.001004,0.842773,0.031525,0.816406,0.005692,0.813965,0.012138,0.827637,0.006683,0.863281,0.001966,0.863281,0.006027,0.801758,0.039673,0.895020,0.001204,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.

In [75]:
df.isnull().sum()

id               0
Periodo          0
Producto-Tipo    0
Count            0
MontoSignoSum    0
                ..
Lectura-1        0
Lectura-2        0
Lectura-3        0
Lectura-4        0
Lectura-5        0
Length: 145, dtype: int64

# Entrenamos el Modelo

In [20]:
df["Periodo"].unique()

array(['2019-04-01T00:00:00.000000000', '2019-07-01T00:00:00.000000000',
       '2019-10-01T00:00:00.000000000', '2020-01-01T00:00:00.000000000',
       '2020-04-01T00:00:00.000000000', '2020-07-01T00:00:00.000000000',
       '2020-10-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [21]:
# split the data into train, validation and test dataset
train_index = sorted(list(df["Periodo"].unique()))[:-1]

test_index = [sorted(list(df["Periodo"].unique()))[-1]]

In [22]:
train_index

[numpy.datetime64('2019-04-01T00:00:00.000000000'),
 numpy.datetime64('2019-07-01T00:00:00.000000000'),
 numpy.datetime64('2019-10-01T00:00:00.000000000'),
 numpy.datetime64('2020-01-01T00:00:00.000000000'),
 numpy.datetime64('2020-04-01T00:00:00.000000000'),
 numpy.datetime64('2020-07-01T00:00:00.000000000')]

In [23]:
test_index

[numpy.datetime64('2020-10-01T00:00:00.000000000')]

In [26]:
variables_borrar=['id',
'Periodo',
'Producto-Tipo',
'Count','MontoSignoSum','MontoSignoMean','MontoSignoMax','MontoSignoMin',
'Count-campana',          
'Canal_B',                 
'Canal_C',                
'Duracion_campana',          
'Resultado',
'Count-comu',
'Lectura',
'Comuna' ,'Ciudad','Profesion','Estado_civil','Principalidad','Meses_antiguedad','Segmento_consumidor',
                  'Edad','Renta','NumProductos','NumProductosGeneral','LecturaMean','LecturaMax','LecturaMin' ]

In [27]:
X_test = df[df["Periodo"].isin(test_index)].drop(variables_borrar, axis = 1)
Y_test = df[df["Periodo"].isin(test_index)]['Count']
X_test_head = df[['id','Periodo','Producto-Tipo','Count']][df["Periodo"].isin(test_index)]

In [28]:
# Rebalanceamos el dataset
# Solo los registros de los clientes que han comprado alguno de los 5 productos en cuestion 
df.drop(df[((df['Producto-Tipo_A-A']==1) & (df['Count']==0)) |
           ((df['Producto-Tipo_B-B']==1) & (df['Count']==0)) |
           ((df['Producto-Tipo_C-D']==1) & (df['Count']==0)) |
           ((df['Producto-Tipo_D-E']==1) & (df['Count']==0)) |
           ((df['Producto-Tipo_E-E']==1) & (df['Count']==0))].index ,  axis=0, inplace=True)

In [29]:
X_train = df[df["Periodo"].isin(train_index)].drop(variables_borrar, axis=1)
Y_train = df[df["Periodo"].isin(train_index)]['Count']


In [30]:
dt = DecisionTreeClassifier(max_depth=10)

In [31]:
dt.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [32]:
score_train=dt.score(X_train, Y_train)

In [87]:
score_train

0.8689097955978788

In [33]:
score_train

0.8689097955978788

In [88]:
y_test_pred = pd.DataFrame(dt.predict(X_test), index=Y_test.index, columns=['altaPrediction'])

In [89]:
results_df = Y_test.to_frame().join(y_test_pred)

In [90]:
results_df

,Count,altaPrediction
7635744,1.0,1.0
7635745,1.0,0.0
7635746,1.0,1.0
7635747,1.0,1.0
7635748,1.0,1.0
...,...,...
8908363,1.0,0.0
8908364,1.0,0.0
8908365,0.0,1.0
8908366,0.0,1.0


In [91]:
X_test_head = pd.concat([X_test_head,results_df['altaPrediction'].to_frame()],axis=1)

In [92]:
X_test_head

,id,Periodo,Producto-Tipo,Count,altaPrediction
7635744,1,2020-10-01,A-K,1.0,1.0
7635745,1,2020-10-01,F-D,1.0,0.0
7635746,1,2020-10-01,F-H,1.0,1.0
7635747,1,2020-10-01,F-J,1.0,1.0
7635748,1,2020-10-01,G-K,1.0,1.0
...,...,...,...,...,...
8908363,79539,2020-10-01,C-C,1.0,0.0
8908364,79539,2020-10-01,E-F,1.0,0.0
8908365,79539,2020-10-01,B-B,0.0,1.0
8908366,79539,2020-10-01,A-A,0.0,1.0


In [93]:
comprobacion=X_test_head[ (X_test_head['Producto-Tipo']=='A-A') |
              (X_test_head['Producto-Tipo']=='B-B') |
              (X_test_head['Producto-Tipo']=='C-D') |
              (X_test_head['Producto-Tipo']=='D-E') |
              (X_test_head['Producto-Tipo']=='E-E') ]

In [94]:
comprobacion

,id,Periodo,Producto-Tipo,Count,altaPrediction
7635752,1,2020-10-01,D-E,0.0,1.0
7635753,1,2020-10-01,C-D,0.0,1.0
7635754,1,2020-10-01,E-E,0.0,1.0
7635757,1,2020-10-01,B-B,0.0,1.0
7635758,1,2020-10-01,A-A,0.0,1.0
...,...,...,...,...,...
8908360,79539,2020-10-01,D-E,0.0,1.0
8908361,79539,2020-10-01,C-D,0.0,1.0
8908362,79539,2020-10-01,E-E,0.0,1.0
8908365,79539,2020-10-01,B-B,0.0,1.0


In [95]:
comprobacion['altaPrediction'].value_counts()

1.0    395070
0.0      2625
Name: altaPrediction, dtype: int64

In [96]:
comprobacion.loc[comprobacion['altaPrediction']>0,
                 'Resultado']=comprobacion[comprobacion['altaPrediction']>0]['Producto-Tipo']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [97]:
comprobacion

,id,Periodo,Producto-Tipo,Count,altaPrediction,Resultado
7635752,1,2020-10-01,D-E,0.0,1.0,D-E
7635753,1,2020-10-01,C-D,0.0,1.0,C-D
7635754,1,2020-10-01,E-E,0.0,1.0,E-E
7635757,1,2020-10-01,B-B,0.0,1.0,B-B
7635758,1,2020-10-01,A-A,0.0,1.0,A-A
...,...,...,...,...,...,...
8908360,79539,2020-10-01,D-E,0.0,1.0,D-E
8908361,79539,2020-10-01,C-D,0.0,1.0,C-D
8908362,79539,2020-10-01,E-E,0.0,1.0,E-E
8908365,79539,2020-10-01,B-B,0.0,1.0,B-B


In [98]:
comprobacion[comprobacion['altaPrediction']>0]

,id,Periodo,Producto-Tipo,Count,altaPrediction,Resultado
7635752,1,2020-10-01,D-E,0.0,1.0,D-E
7635753,1,2020-10-01,C-D,0.0,1.0,C-D
7635754,1,2020-10-01,E-E,0.0,1.0,E-E
7635757,1,2020-10-01,B-B,0.0,1.0,B-B
7635758,1,2020-10-01,A-A,0.0,1.0,A-A
...,...,...,...,...,...,...
8908360,79539,2020-10-01,D-E,0.0,1.0,D-E
8908361,79539,2020-10-01,C-D,0.0,1.0,C-D
8908362,79539,2020-10-01,E-E,0.0,1.0,E-E
8908365,79539,2020-10-01,B-B,0.0,1.0,B-B


In [99]:
comprobacion.fillna('',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [100]:
comprobacion[comprobacion['altaPrediction']>0]['Producto-Tipo'].value_counts()

A-A    79482
B-B    79456
E-E    79352
D-E    78959
C-D    77821
Name: Producto-Tipo, dtype: int64

In [101]:
comprobacion_=comprobacion.groupby(['id','Periodo']).agg({'Resultado':np.sum})

In [102]:
comprobacion_

,,Resultado
id,Periodo,
1,2020-10-01,D-EC-DE-EB-BA-A
2,2020-10-01,D-EC-DE-EB-BA-A
3,2020-10-01,D-EC-DE-EB-BA-A
4,2020-10-01,D-EC-DE-EB-BA-A
5,2020-10-01,D-EC-DE-EB-BA-A
...,...,...
79535,2020-10-01,D-EC-DE-EB-BA-A
79536,2020-10-01,D-EC-DE-EB-BA-A
79537,2020-10-01,D-EC-DE-EB-BA-A


In [103]:
comprobacion_['Resultado'].value_counts()

D-EC-DE-EB-BA-A    77756
D-EE-EB-BA-A         979
E-EB-BA-A            447
D-EB-BA-A            120
D-EE-EA-A             53
C-DE-EB-BA-A          48
B-BA-A                45
D-EE-EB-B             31
E-EA-A                15
E-EB-B                11
D-EC-DB-BA-A           5
D-EE-E                 4
A-A                    4
C-DB-BA-A              4
D-EB-B                 4
D-EA-A                 3
B-B                    2
C-DE-EA-A              2
C-DE-EB-B              2
D-EC-DE-EB-B           2
D-EC-DE-EA-A           1
D-EC-DE-E              1
Name: Resultado, dtype: int64

In [104]:
comprobacion_['Resultado'].replace('D-EA-A','D-E A-A',inplace=True)
comprobacion_['Resultado'].replace('D-EE-E','D-E E-E',inplace=True)
comprobacion_['Resultado'].replace('D-EE-EA-A','D-E E-E A-A',inplace=True)
comprobacion_['Resultado'].replace('E-EA-A','E-E A-A',inplace=True)
comprobacion_['Resultado'].replace('D-EC-DE-E','D-E C-D E-E',inplace=True)
comprobacion_['Resultado'].replace('C-DE-E','C-D E-E',inplace=True)
comprobacion_['Resultado'].replace('D-EC-D','D-E C-D',inplace=True)
comprobacion_['Resultado'].replace('D-EC-DE-EA-A','D-E C-D E-E A-A',inplace=True)
comprobacion_['Resultado'].replace('C-DE-EA-A','C-D E-E A-A',inplace=True)
comprobacion_['Resultado'].replace('D-EC-DA-A','D-E C-D A-A',inplace=True)
comprobacion_['Resultado'].replace('C-DA-A','C-D A-A',inplace=True)
comprobacion_['Resultado'].replace('D-EB-B','D-E B-B',inplace=True)

comprobacion_['Resultado'].replace('C-DE-EB-B','C-D E-E B-B',inplace=True)
comprobacion_['Resultado'].replace('D-EE-EB-B','D-E E-E B-B',inplace=True)
comprobacion_['Resultado'].replace('D-EC-DE-EB-B','D-E C-D E-E B-B',inplace=True)
comprobacion_['Resultado'].replace('D-EC-DB-B','D-E C-D B-B',inplace=True)
comprobacion_['Resultado'].replace('D-EC-DE-EB-BA-A','D-E C-D E-E B-B A-A',inplace=True)
comprobacion_['Resultado'].replace('C-DE-EB-BA-A','C-D E-E B-B A-A',inplace=True)

comprobacion_['Resultado'].replace('D-EB-BA-A','D-E B-B A-A',inplace=True)
comprobacion_['Resultado'].replace('B-BA-A','B-B A-A',inplace=True)
comprobacion_['Resultado'].replace('D-EE-EB-BA-A','D-E E-E B-B A-A',inplace=True)

comprobacion_['Resultado'].replace('C-DB-B','C-D B-B',inplace=True)
comprobacion_['Resultado'].replace('D-EC-DB-BA-A','D-E C-D B-B A-A',inplace=True)
comprobacion_['Resultado'].replace('E-EB-B','E-E B-B',inplace=True)
comprobacion_['Resultado'].replace('C-DB-BA-A','C-D B-B A-A',inplace=True)
comprobacion_['Resultado'].replace('E-EB-BA-A','E-E B-B A-A',inplace=True)


In [105]:
comprobacion_['Resultado'].value_counts()

D-E C-D E-E B-B A-A    77756
D-E E-E B-B A-A          979
E-E B-B A-A              447
D-E B-B A-A              120
D-E E-E A-A               53
C-D E-E B-B A-A           48
B-B A-A                   45
D-E E-E B-B               31
E-E A-A                   15
E-E B-B                   11
D-E C-D B-B A-A            5
D-E B-B                    4
D-E E-E                    4
A-A                        4
C-D B-B A-A                4
D-E A-A                    3
B-B                        2
C-D E-E A-A                2
C-D E-E B-B                2
D-E C-D E-E B-B            2
D-E C-D E-E                1
D-E C-D E-E A-A            1
Name: Resultado, dtype: int64

In [106]:
comprobacion_.reset_index(drop=False,inplace=True)

In [107]:
comprobacion_.columns=['id','Periodo','productos']

In [108]:
comprobacion_

,id,Periodo,productos
0,1,2020-10-01,D-E C-D E-E B-B A-A
1,2,2020-10-01,D-E C-D E-E B-B A-A
2,3,2020-10-01,D-E C-D E-E B-B A-A
3,4,2020-10-01,D-E C-D E-E B-B A-A
4,5,2020-10-01,D-E C-D E-E B-B A-A
...,...,...,...
79534,79535,2020-10-01,D-E C-D E-E B-B A-A
79535,79536,2020-10-01,D-E C-D E-E B-B A-A
79536,79537,2020-10-01,D-E C-D E-E B-B A-A
79537,79538,2020-10-01,D-E C-D E-E B-B A-A


In [109]:
comprobacion_[['id','productos']].to_csv('prediccion-08122020.csv',index=False)

In [112]:
top_features = pd.Series(dt.feature_importances_, index=X_train.columns).sort_values(ascending=False).head(40)

In [113]:
top_features

Count-1                              0.571141
MontoSignoSum-1                      0.115520
Count-comu-1                         0.102481
Principalidad_Meanratio              0.053220
Count-2                              0.038199
EdadNumber_Meanratio                 0.017802
Sexo_Meanratio                       0.016201
Producto-Tipo_D-E                    0.010845
MontoSignoMax-1                      0.009582
NumProductos-1                       0.009436
Count-5                              0.008519
Profesion_Meanratio                  0.006221
Producto-Tipo_C-D                    0.005661
month                                0.004483
Recibe_sueldo_en_cuenta_Meanratio    0.003775
Count-3                              0.003658
Ciudad_Meanratio                     0.003181
NumProductosGeneral-2                0.002916
MontoSignoMax-4                      0.001275
MontoSignoMean-4                     0.001236
Segmento_consumidorNum_Meanratio     0.001139
Producto-Tipo_C-C                 

# Entrenamos el modelo pruebas

In [49]:
df=pd.read_pickle('./3T/df_trans_campanas_comu_3M_V6.pkl',compression='zip')

In [50]:
# split the data into train, validation and test dataset
train_index = sorted(list(df["Periodo"].unique()))[:-2]

test_index = [sorted(list(df["Periodo"].unique()))[-2]]

In [51]:
train_index

[numpy.datetime64('2019-04-01T00:00:00.000000000'),
 numpy.datetime64('2019-07-01T00:00:00.000000000'),
 numpy.datetime64('2019-10-01T00:00:00.000000000'),
 numpy.datetime64('2020-01-01T00:00:00.000000000'),
 numpy.datetime64('2020-04-01T00:00:00.000000000')]

In [52]:
test_index

[numpy.datetime64('2020-07-01T00:00:00.000000000')]

In [54]:
variables_borrar=['id',
'Periodo',
'Producto-Tipo',
'Count','MontoSignoSum','MontoSignoMean','MontoSignoMax','MontoSignoMin',
'Count-campana',          
'Canal_B',                 
'Canal_C',                
'Duracion_campana',          
'Resultado',
'Count-comu',
'Lectura',
'Comuna' ,'Ciudad','Profesion','Estado_civil','Principalidad','Meses_antiguedad','Segmento_consumidor',
                  'Edad','Renta','NumProductos','NumProductosGeneral' ]

In [55]:
X_test = df[df["Periodo"].isin(test_index)].drop(variables_borrar, axis = 1)
Y_test = df[df["Periodo"].isin(test_index)]['Count']
X_test_head = df[['id','Periodo','Producto-Tipo','Count']][df["Periodo"].isin(test_index)]

In [56]:
# Rebalanceamos el dataset
df.drop(df[((df['Producto-Tipo_A-A']==1) & (df['Count']==0)) |
           ((df['Producto-Tipo_B-B']==1) & (df['Count']==0)) |
           ((df['Producto-Tipo_C-D']==1) & (df['Count']==0)) |
           ((df['Producto-Tipo_D-E']==1) & (df['Count']==0)) |
           ((df['Producto-Tipo_E-E']==1) & (df['Count']==0))].index ,  axis=0, inplace=True)

In [57]:
X_train = df[df["Periodo"].isin(train_index)].drop(variables_borrar, axis=1)
Y_train = df[df["Periodo"].isin(train_index)]['Count']

In [58]:
dt = DecisionTreeClassifier(max_depth=12)

In [59]:
dt.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [60]:
score_train=dt.score(X_train, Y_train)
score_test=dt.score(X_test, Y_test)

In [61]:
print('Score_train: {} Score_test: {}'.format(score_train,score_test))

Score_train: 0.8657584380659266 Score_test: 0.6411948855278543


In [62]:
dt = DecisionTreeClassifier(max_depth=16)

In [63]:
dt.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=16, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [64]:
score_train=dt.score(X_train, Y_train)
score_test=dt.score(X_test, Y_test)
print('Score_train: {} Score_test: {}'.format(score_train,score_test))

Score_train: 0.8731834155283515 Score_test: 0.6360653264436315


In [65]:
dt = DecisionTreeClassifier(max_depth=8)

In [66]:
dt.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [67]:
score_train=dt.score(X_train, Y_train)
score_test=dt.score(X_test, Y_test)
print('Score_train: {} Score_test: {}'.format(score_train,score_test))

Score_train: 0.8603169392865371 Score_test: 0.6452510718012547


In [68]:
dt = DecisionTreeClassifier(max_depth=6)

In [69]:
dt.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [70]:
score_train=dt.score(X_train, Y_train)
score_test=dt.score(X_test, Y_test)
print('Score_train: {} Score_test: {}'.format(score_train,score_test))

Score_train: 0.8554164283487954 Score_test: 0.6364385710154767


In [71]:
dt = DecisionTreeClassifier(max_depth=10)

In [72]:
dt.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [73]:
score_train=dt.score(X_train, Y_train)
score_test=dt.score(X_test, Y_test)
print('Score_train: {} Score_test: {}'.format(score_train,score_test))

Score_train: 0.8635531866334918 Score_test: 0.645545738568501
